This is a code to find Major Merger Galaxy Pairs from the large scale hydrodynamic cosmological simulation IllustrisTNG. Two verisons of TNG are used here: TNG100-1 and TNG300-1. The data sets read originally contain tens of millions of galaxies, but the files have been greatly reduced by eliminated galaxies whose stellar mass is less then 0.4*10^10 solar masses. - Spencer Shortt

In [26]:
import pandas as pd
import numpy as np
import MajorMergerFunctions as mmf
import time
import matplotlib.pyplot as plt
import webbrowser
%matplotlib inline

Choose Data file to read from (100 or 300):

In [27]:
#data = pd.read_csv('/Users/spencershortt/TNG2019/TNG_100_data.txt', sep="\t")
data = pd.read_csv('/Users/spencershortt/TNG2019/TNG_300_data.txt', sep="\t")

In [28]:
df=pd.DataFrame(data)

In [29]:
df.head()

,id,mass_stars,pos_x,pos_y,pos_z,vel_x,vel_y,vel_z,stellarphotometrics_k,sfr,GasMass
0,0,530.9480,43718.8,48813.6,147595.0,472.196,450.850,-260.747,-28.9114,0.0,19187.50
1,1,253.6630,45442.3,51850.2,146416.0,-209.057,-735.889,400.642,-28.1718,0.0,5069.83
2,2,57.3657,44490.8,49091.7,147871.0,2021.730,1495.440,-1797.080,-26.5537,0.0,0.00
3,3,23.9098,43820.8,50939.4,147711.0,925.150,-473.445,-275.926,-25.6203,0.0,0.00
4,4,22.0596,44302.6,49631.0,147869.0,-260.215,-2221.630,-563.641,-25.5180,0.0,0.00


In [30]:
df.tail()

,id,mass_stars,pos_x,pos_y,pos_z,vel_x,vel_y,vel_z,stellarphotometrics_k,sfr,GasMass
129918,2904582,0.497388,34488.9,44112.2,142250.0,-253.2380,535.2530,1091.7900,-21.5323,0.0,0.0
129919,2905963,0.496814,162828.0,38293.6,186559.0,-35.2799,76.5991,-323.0800,-21.6264,0.0,0.0
129920,2907383,0.406907,172186.0,174281.0,104841.0,390.4080,71.0224,-50.0765,-21.2487,0.0,0.0
129921,2963127,0.491392,39815.6,98998.8,110477.0,-109.3380,275.9260,148.5560,-21.4357,0.0,0.0
129922,3254912,0.457672,47069.8,52358.7,144897.0,-331.3060,-342.6480,802.9270,-21.4755,0.0,0.0


Establish conditions to find Major Merger Galaxy Pairs. There is 2d and 3d criteria that is applied separately, giving me two different groups of major merger pairs.

In [31]:
#Initial Conditions

mass_cutOff=1.0

#Observational:
o_kband_cutOff=1.0
o_radius_cutOff=20.0
o_velocity_cutOff=1000.0
#3d:
threeD_massRatio_high_cutOff=2.5
threeD_massRatio_low_cutOff=0.4
threeD_radius_cutOff=20.0
threeD_velocity_cutOff=1000

Initialize our two new data frames. One dataframe contains galaxies who belong to a major merger pair by using 2d criteria, while the other is for 3d criteria.

In [32]:
observe_pairs=pd.DataFrame([],columns=df.columns)
observe_pairs["Pair_IDs"]=""
observe_pairs["Separation"]=""
observe_pairs["Rel_v_z"]=""
observe_pairs["Delta_k"]=""

threeD_pairs=pd.DataFrame([],columns=df.columns)
threeD_pairs["Pair_IDs"]=""
threeD_pairs["Separation"]=""
threeD_pairs["Rel_v"]=""
threeD_pairs["Mass_Ratio"]=""

This is the cell the applies the criteria. Each galaxy is checked against every other galaxy in our data to see if they count as major merger pairs according to the 2d and 3d criteria. Afterwards, the data are saved to a txt file for further analysis. Note - there will sometimes be triples or groups; these are eliminated in a different code.

In [33]:
#start_time = time.time()



#Loop through each galaxy
for id in df["id"]:
    #Create dataframe for this particular galaxy
    galaxy=df.loc[df["id"]==id]
    
    if galaxy.mass_stars.item()>mass_cutOff:
        
        #Calculate Observational parameters
        delta_k=mmf.delta_k(galaxy.stellarphotometrics_k.item(),df.stellarphotometrics_k.values)
        r_2d=mmf.distance_2d(galaxy.pos_x.item(),galaxy.pos_y.item(),df.pos_x.values,df.pos_y.values)
        rel_v_z=mmf.rel_v_z(galaxy.vel_z.item(),df.vel_z.values)
        
        #Calculate the boolean arrays based on initial parameters - Observational
        k_condition_2d=delta_k<o_kband_cutOff
        r_condition_2d=r_2d<o_radius_cutOff
        v_condition_2d=rel_v_z<o_velocity_cutOff
        
        #Find Observational Pairs!
        pair_2d=df[(k_condition_2d) & (r_condition_2d) & (v_condition_2d)]#&(id_condition)]
        
        #Need to initially keep triples, then remove everything that is triple
        if len(pair_2d.index)>1:
            #pairIds=""
            
            #for i,I in enumerate(pair_2d.id.values):
            #    pairIds+=str(I)+","
            pairIds=','.join(map(str,pair_2d.id.values )) 
                
            pair_2d["Pair_IDs"]=pairIds
            
            if len(pair_2d.index)==2:
                pairSep_2d=mmf.distance_2d(pair_2d.iloc[0].loc["pos_x"],
                                    pair_2d.iloc[0].loc["pos_y"],
                                    pair_2d.iloc[1].loc["pos_x"],
                                    pair_2d.iloc[1].loc["pos_y"])
            
                pairRelV_2d=mmf.rel_v_z(pair_2d.iloc[0].loc["vel_z"],pair_2d.iloc[1].loc["vel_z"])
                
                pairDelta_k_2d=mmf.delta_k(pair_2d.iloc[0].loc["stellarphotometrics_k"],
                                           pair_2d.iloc[1].loc["stellarphotometrics_k"])
            
                pair_2d["Delta_k"]=[pairDelta_k_2d,pairDelta_k_2d]
                pair_2d["Separation"]=[pairSep_2d, pairSep_2d]
                pair_2d["Rel_v_z"]=[pairRelV_2d,pairRelV_2d]
        
            observe_pairs=pd.concat([observe_pairs,pair_2d]).drop_duplicates().reset_index(drop=True)
            #counter+=1
            #print(counter)
            
        #Calculate 3d parameters:
        mass_ratio=mmf.mass_ratio(galaxy.mass_stars.item(), df.mass_stars.values)
        r_3d=mmf.distance_3d(galaxy.pos_x.item(),galaxy.pos_y.item(),galaxy.pos_z.item(),
                                 df.pos_x.values,df.pos_y.values,df.pos_z.values)
        rel_v=mmf.rel_v_3d(galaxy.vel_x.item(),galaxy.vel_y.item(),galaxy.vel_z.item(),
                                 df.vel_x.values,df.vel_y.values,df.vel_z.values)
         
            
            
            
            
        #Calculate the boolean arrays based on initial parameters - 3d
        mr_condition_3d=(mass_ratio>threeD_massRatio_low_cutOff)&(mass_ratio<threeD_massRatio_high_cutOff)
        r_condition_3d=r_3d<threeD_radius_cutOff
        v_condition_3d=rel_v<threeD_velocity_cutOff
            
        #Find 3d Pairs!
        pair_3d=df[(mr_condition_3d)&(r_condition_3d)&(v_condition_3d)]
            
        #Need to initially keep triples, then remove everything that is triple
        if len(pair_3d.index)>1:
        
            pairIds=','.join(map(str,pair_3d.id.values ))

            pair_3d["Pair_IDs"]=pairIds

            if len(pair_3d.index)==2:
                pairSep_3d=mmf.distance_3d(pair_3d.iloc[0].loc["pos_x"],
                                        pair_3d.iloc[0].loc["pos_y"],
                                        pair_3d.iloc[0].loc["pos_z"],
                                        pair_3d.iloc[1].loc["pos_x"],
                                        pair_3d.iloc[1].loc["pos_y"],
                                        pair_3d.iloc[1].loc["pos_z"])

                pairRelV_3d=mmf.rel_v_3d(pair_3d.iloc[0].loc["vel_x"],
                                             pair_3d.iloc[0].loc["vel_y"],
                                             pair_3d.iloc[0].loc["vel_z"],
                                             pair_3d.iloc[1].loc["vel_x"],
                                             pair_3d.iloc[1].loc["vel_y"],
                                             pair_3d.iloc[1].loc["vel_z"])
                
                pairMassRat_3d=mmf.mass_ratio(max(pair_3d.iloc[0].loc["mass_stars"],
                                                  pair_3d.iloc[1].loc["mass_stars"]),
                                              min(pair_3d.iloc[0].loc["mass_stars"],
                                                  pair_3d.iloc[1].loc["mass_stars"]))

                pair_3d["Separation"]=[pairSep_3d, pairSep_3d]
                pair_3d["Rel_v"]=[pairRelV_3d,pairRelV_3d]
                pair_3d["Mass_Ratio"]=[pairMassRat_3d,pairMassRat_3d]

            threeD_pairs=pd.concat([threeD_pairs,pair_3d]).drop_duplicates().reset_index(drop=True)
            
        
#print("--- %s seconds ---" % (time.time() - start_time))
'''       
observe_pairs=observe_pairs.dropna()
observe_pairs=observe_pairs.reset_index()
threeD_pairs=threeD_pairs.dropna()
threeD_pairs=threeD_pairs.reset_index()
'''


observe_pairs["sSFR"]=observe_pairs.sfr.values/observe_pairs.mass_stars.values
threeD_pairs["sSFR"]=threeD_pairs.sfr.values/threeD_pairs.mass_stars.values


#observe_pairs.to_csv('/Users/spencershortt/TNG2019/observe_pairs_100.txt', sep="\t")
#threeD_pairs.to_csv('/Users/spencershortt/TNG2019/threeD_pairs_100.txt', sep="\t")
observe_pairs.to_csv('/Users/spencershortt/TNG2019/observe_pairs_300.txt', sep="\t")
threeD_pairs.to_csv('/Users/spencershortt/TNG2019/threeD_pairs_300.txt', sep="\t")

observe_pairs.head(10)

/Users/spencershortt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/spencershortt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/spencershortt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydat

,Delta_k,GasMass,Pair_IDs,Rel_v_z,Separation,id,mass_stars,pos_x,pos_y,pos_z,sfr,stellarphotometrics_k,vel_x,vel_y,vel_z,sSFR
0,0.4686,0.0,"138,218",311.612,16.493938,138,3.921860,43895.2,48848.1,147578.0,0.0,-23.6853,-65.7513,828.8280,-380.372,0.0
1,0.4686,0.0,"138,218",311.612,16.493938,218,2.522540,43898.3,48831.9,147765.0,0.0,-23.2167,-1488.1100,-429.3420,-691.984,0.0
2,0.8767,0.0,"158,330",322.560,1.811077,158,3.721980,43734.2,48916.5,147545.0,0.0,-23.5634,-264.2120,-102.8070,-1630.210,0.0
3,0.8767,0.0,"158,330",322.560,1.811077,330,1.474030,43734.0,48918.3,147555.0,0.0,-22.6867,-268.2300,1558.5200,-1307.650,0.0
4,0.5663,0.0,"165,264",337.789,8.711487,165,3.082250,45533.7,51898.1,146308.0,0.0,-23.5271,961.0520,-1184.1700,181.010,0.0
5,0.5663,0.0,"165,264",337.789,8.711487,264,1.999310,45540.2,51892.3,146431.0,0.0,-22.9608,-684.7630,-387.1800,518.799,0.0
6,0.5752,0.0,"320,380",409.986,19.498718,320,0.606595,45345.7,52013.0,146501.0,0.0,-21.7214,2059.4300,-1308.0300,-212.605,0.0
7,0.5752,0.0,"320,380",409.986,19.498718,380,1.068020,45354.5,51995.6,146345.0,0.0,-22.2966,-665.9410,-109.0500,-622.591,0.0
8,0.5313,0.0,"11810,11812",265.339,18.104143,11810,4.225830,81961.4,121109.0,194317.0,0.0,-23.8580,501.5270,58.5497,-446.784,0.0
9,0.5313,0.0,"11810,11812",265.339,18.104143,11812,2.841550,81978.8,121104.0,194458.0,0.0,-23.3267,-2278.3700,2017.9200,-181.445,0.0


In [34]:
threeD_pairs

,GasMass,Mass_Ratio,Pair_IDs,Rel_v,Separation,id,mass_stars,pos_x,pos_y,pos_z,sfr,stellarphotometrics_k,vel_x,vel_y,vel_z,sSFR
0,1.204250,2.270149,"22786,22884",56.750755,18.690372,22786,2.056680,86832.20,81637.1,52032.10,0.173751,-23.5536,-1081.740000,-723.5290,1848.1600,0.084481
1,0.558519,2.270149,"22786,22884",56.750755,18.690372,22884,0.905967,86832.20,81619.3,52026.40,5.775790,-22.8834,-1086.450000,-667.7290,1857.3700,6.375276
2,2.160950,2.180896,"140489,140547",211.156688,19.533049,140489,1.261910,83840.50,35332.9,14348.10,5.402780,-23.3781,-846.969000,1080.5600,124.1250,4.281431
3,0.234724,2.180896,"140489,140547",211.156688,19.533049,140547,0.578620,83848.80,35337.0,14330.90,2.712000,-22.8839,-685.602000,1209.0700,169.2140,4.687014
4,0.000000,1.886368,"159056,159069",926.399468,17.535393,159056,5.339270,83474.70,18256.5,143789.00,0.000000,-24.1348,39.465100,-285.3710,-319.8750,0.000000
5,0.000000,1.886368,"159056,159069",926.399468,17.535393,159069,2.830450,83465.40,18267.5,143799.00,0.000000,-23.2674,817.543000,40.4644,63.0637,0.000000
6,0.000000,NaN,"159056,159069,159084",NaN,NaN,159056,5.339270,83474.70,18256.5,143789.00,0.000000,-24.1348,39.465100,-285.3710,-319.8750,0.000000
7,0.000000,NaN,"159056,159069,159084",NaN,NaN,159069,2.830450,83465.40,18267.5,143799.00,0.000000,-23.2674,817.543000,40.4644,63.0637,0.000000
8,0.000000,NaN,"159056,159069,159084",NaN,NaN,159084,1.689610,83476.40,18280.8,143805.00,0.000000,-23.0089,378.584000,-502.3940,130.1250,0.000000
9,0.000000,1.675209,"159069,159084",701.339764,18.272657,159069,2.830450,83465.40,18267.5,143799.00,0.000000,-23.2674,817.543000,40.4644,63.0637,0.000000
